In [1]:
import numpy as np
import pandas as pd
import math

# Inferential Statistics - Confidence Intervals

In [2]:
dataset = pd.read_csv('ex3-17.csv').rename(columns={'Unnamed: 11': 'Year'})
dataset

,InvoiceNo,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice
0,52389,01/01/2014,United Kingdom,2152,UK2,Male,11.0,44,10.5,£159.00,0%,2014,1,£159.00
1,52390,01/01/2014,United States,2230,US15,Male,11.5,44-45,11.0,£199.00,20%,2014,1,£159.20
2,52391,01/01/2014,Canada,2160,CAN7,Male,9.5,42-43,9.0,£149.00,20%,2014,1,£119.20
3,52392,01/01/2014,United States,2234,US6,Female,9.5,40,7.5,£159.00,0%,2014,1,£159.00
4,52393,01/01/2014,United Kingdom,2222,UK4,Female,9.0,39-40,7.0,£159.00,0%,2014,1,£159.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14962,65773,31/12/2016,United Kingdom,2154,UK2,Male,9.5,42-43,9.0,£139.00,0%,2016,12,£139.00
14963,65774,31/12/2016,United States,2181,US12,Female,12.0,42-43,10.0,£149.00,0%,2016,12,£149.00
14964,65775,31/12/2016,Canada,2203,CAN6,Male,10.5,43-44,10.0,£179.00,30%,2016,12,£125.30
14965,65776,31/12/2016,Germany,2231,GER1,Female,9.5,40,7.5,£199.00,0%,2016,12,£199.00


**Task 1:** Calculate the confidence intervals for men shoes sales in the USA, this time based on a bigger sample - 2015-2016.

In [3]:
men_2015 = pd.DataFrame(columns=np.arange(1, 13), index=pd.unique(dataset['Size (US)'])).fillna(0).sort_index()
men_2016 = pd.DataFrame(columns=np.arange(1, 13), index=pd.unique(dataset['Size (US)'])).fillna(0).sort_index()

for i, row in dataset.iterrows():
    if row['Gender'] == 'Male' and row['Country'] == 'United States':
        if int(row['Size (US)']) >= 6:
            if row['Year'] == 2015:
                men_2015.at[row['Size (US)'], row['Month']] += 1
            if row['Year'] == 2016:
                men_2016.at[row['Size (US)'], row['Month']] += 1

In [4]:
joined = pd.concat([men_2015, men_2016], axis=1, sort=False)
joined = joined.drop(index=[4.5, 5.0, 5.5])
joined['Mean'] = joined.mean(axis=1)

men_2016 = men_2016.drop(index=[4.5, 5.0, 5.5])
men_2016['Mean'] = men_2016.mean(axis=1)

In [5]:
joined['Std Error'] = joined.std(axis=1) / math.sqrt(24)
men_2016['Std Error'] = men_2016.std(axis=1) / math.sqrt(12)

In [6]:
t_value = 2.074
joined['Margin of Error'] = joined['Std Error'] * t_value

t_value = 2.201
men_2016['Margin of Error'] = men_2016['Std Error'] * t_value

In [7]:
joined["95% Low"] = joined['Mean'] - joined['Margin of Error']
joined["95% High"] = joined['Mean'] + joined['Margin of Error']

men_2016['95% High'] = men_2016['Mean'] + men_2016['Margin of Error']



**Task 2:** Compare the results with the confidence intervals when the sample was based just on 2016.

In [8]:
joined['Max Pairs'] = joined['95% High'].apply(np.ceil)
men_2016['Max Pairs'] = men_2016['95% High'].apply(np.ceil)

diff = np.abs(joined['Max Pairs'] - men_2016['Max Pairs'])
diff

6.0     2.0
6.5     0.0
7.0     0.0
7.5     1.0
8.0     1.0
8.5     4.0
9.0     4.0
9.5     6.0
10.0    5.0
10.5    3.0
11.0    2.0
11.5    1.0
12.0    1.0
13.0    1.0
14.0    1.0
15.0    1.0
Name: Max Pairs, dtype: float64

**Task 3:** Estimate the 90% confidence interval for the same task as the lesson. What changes can you see?

In [9]:
ger1 = pd.DataFrame(columns=np.arange(1, 13), index=pd.unique(dataset['Size (US)'])).fillna(0).sort_index()
ger2 = pd.DataFrame(columns=np.arange(1, 13), index=pd.unique(dataset['Size (US)'])).fillna(0).sort_index()

for i, row in dataset.iterrows():
    if row['Gender'] == 'Female' and row['Year'] == 2016:
        if row['Shop'] == 'GER1':
            ger1.at[row['Size (US)'], row['Month']] += 1
        if row['Shop'] == 'GER2':
            ger2.at[row['Size (US)'], row['Month']] += 1

In [10]:
ger1['Mean'] = ger1.mean(axis=1)
ger2['Mean'] = ger2.mean(axis=1)

In [11]:
ger1['Sample Variance'] = ger1.var(axis=1)
ger2['Sample Variance'] = ger2.var(axis=1)

In [12]:
pooled_variance = (ger1['Sample Variance'] * 11 + ger2['Sample Variance'] * 11) / 22

In [13]:
t_value = 1.717
margin_of_error = t_value * np.sqrt(2 * (pooled_variance / 12))

In [14]:
confidence_interval = pd.DataFrame(columns=['Low', 'High'])
confidence_interval['Low'] = ger1['Mean'] - ger2['Mean'] - margin_of_error
confidence_interval['High'] = ger1['Mean'] - ger2['Mean'] + margin_of_error
confidence_interval

,Low,High
4.5,-0.115349,0.782016
5.0,-0.387471,0.387471
5.5,-0.588093,0.088093
6.0,-0.963075,0.129742
6.5,-0.172665,1.339332
7.0,-1.316001,0.649334
7.5,-4.257619,0.090953
8.0,-3.599284,1.099284
8.5,-0.273797,3.107130
9.0,-1.701609,1.534943


Both shops perform about equally, with neither selling more or less than the other (90% of the time). The exception is for size 10 shoes, for which GER2 sells more.